In [ ]:
# %%
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, mean_absolute_error, mean_squared_error
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import (
    Conv1D, MaxPooling1D, LSTM, Dropout, LayerNormalization, Dense, TimeDistributed,
    RepeatVector, MultiHeadAttention, Concatenate, Bidirectional
)
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from keras_tuner import Hyperband
import tensorflow.keras.backend as K

# %%
# Carregar dados
data = pd.read_csv("dataset.csv")

# Converter 'id' em datetime e definir como índice
data['timestamp'] = pd.to_datetime(data['id'], errors='coerce')
data.set_index('timestamp', inplace=True)

# Selecionar variáveis relevantes
variables = data[['ws100', 'humid', 'wdisp100', 'hour', 'wdir100']]

# Padronização dos dados usando MinMaxScaler
scaler = MinMaxScaler()
variables_scaled = scaler.fit_transform(variables)

# Parâmetros para sequências de entrada e saída
sequence_length = 36   # Janela de aprendizado de 36 passos de tempo
step_ahead = 6         # Previsão para o sexto passo (1 hora de antecedência)
split_ratio = 0.80     # 80% para treinamento e 20% para teste

# Divisão dos dados em treinamento e teste
split_index = int(len(variables_scaled) * split_ratio)
train_data = variables_scaled[:split_index]
test_data = variables_scaled[split_index:]

# Função para preparar sequências de dados para previsão de um passo específico
def create_sequences_single_ahead(data, seq_length, step_ahead=6):
    X, y = [], []
    for i in range(len(data) - seq_length - step_ahead + 1):
        X.append(data[i:i+seq_length])
        y.append(data[i + seq_length + step_ahead -1, 0])  # ws100 é a primeira coluna
    return np.array(X), np.array(y)

# Criar sequências para treinamento e teste
X_train, y_train = create_sequences_single_ahead(train_data, sequence_length, step_ahead)
X_test, y_test = create_sequences_single_ahead(test_data, sequence_length, step_ahead)

# Reshape de y_train e y_test para serem compatíveis com a saída do modelo
y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

# Valores mínimos e máximos de 'ws100' para inversão da escala
ws100_min = scaler.data_min_[0]
ws100_max = scaler.data_max_[0]

# Número de características
num_features = X_train.shape[2]

# Verificação das formas dos dados
print(f"X_train shape: {X_train.shape}")  # Esperado: (num_samples, 36, 5)
print(f"y_train shape: {y_train.shape}")  # Esperado: (num_samples, 1)
print(f"X_test shape: {X_test.shape}")    # Esperado: (num_samples, 36, 5)
print(f"y_test shape: {y_test.shape}")    # Esperado: (num_samples, 1)")

# %%
# Função de Data Augmentation para amostras com y < 6 m/s
def augment_data(X, y, num_augmented_per_sample=5):
    X_augmented = []
    y_augmented = []
    for i in range(len(y)):
        # Inversão da normalização de y
        y_inv = y[i] * (ws100_max - ws100_min) + ws100_min
        if y_inv < 6.0:
            for _ in range(num_augmented_per_sample):
                noise = np.random.normal(0, 0.01, X[i].shape)
                X_augmented.append(X[i] + noise)
                y_augmented.append(y[i])
    if X_augmented:
        X_augmented = np.array(X_augmented)
        y_augmented = np.array(y_augmented)
        X = np.concatenate((X, X_augmented), axis=0)
        y = np.concatenate((y, y_augmented), axis=0)
    return X, y

# Aplicar data augmentation nos dados de treinamento
X_train, y_train = augment_data(X_train, y_train)

print(f"After augmentation, X_train shape: {X_train.shape}")
print(f"After augmentation, y_train shape: {y_train.shape}")

# %%
# Função de perda personalizada sensível a outliers
def get_custom_loss(ws100_min, ws100_max):
    def custom_loss(y_true, y_pred):
        # Inversão da normalização para obter valores originais
        y_true_inv = y_true * (ws100_max - ws100_min) + ws100_min
        y_pred_inv = y_pred * (ws100_max - ws100_min) + ws100_min

        # Calcula o erro absoluto
        error = K.abs(y_true_inv - y_pred_inv)

        # Cria um tensor booleano indicando onde y_true_inv < 6 m/s
        condition = K.less(y_true_inv, 6.0)

        # Define um peso maior para velocidades abaixo de 6 m/s
        greater_weight = tf.cast(5.0, y_true.dtype)  # Peso maior para erros quando y_true_inv < 6 m/s
        lesser_weight = tf.cast(1.0, y_true.dtype)   # Peso normal para outros casos

        # Aplica os pesos garantindo que tenham a mesma forma que 'error'
        weights = K.switch(condition, greater_weight * K.ones_like(error), lesser_weight * K.ones_like(error))
        # Alternativamente, você pode usar:
        # weights = K.cast(condition, y_true.dtype) * (greater_weight - lesser_weight) + lesser_weight

        weighted_error = weights * error

        # Retorna a média do erro ponderado
        return K.mean(weighted_error)
    
    return custom_loss

# %%
# Função para construir o modelo com atenção e LSTM bidirecional
def build_model_cnn(hp):
    # Hiperparâmetros com espaços de busca reduzidos
    filters = hp.Int('filters', min_value=16, max_value=256, step=16)
    kernel_size = hp.Choice('kernel_size', values=[3, 5, 7])
    pool_size = hp.Choice('pool_size', values=[2, 3])
    units = hp.Int('units', min_value=16, max_value=256, step=16)
    learning_rate = hp.Choice('learning_rate', [1e-4, 1e-3, 1e-2])
    dropout_rate = hp.Float('dropout_rate', min_value=0.2, max_value=0.5, step=0.1)
    optimizer_choice = hp.Choice('optimizer', ['adam', 'rmsprop'])
    num_heads = hp.Int('num_heads', min_value=2, max_value=6, step=2)
    
    # Seleção do Otimizador
    if optimizer_choice == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_choice == 'rmsprop':
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    
    # Definir a função de perda personalizada com ws100_min e ws100_max
    custom_loss = get_custom_loss(ws100_min, ws100_max)
    
    # Entrada do Encoder
    encoder_inputs = tf.keras.Input(shape=(sequence_length, num_features), name='encoder_input')
    x = encoder_inputs

    # Camada CNN
    x = Conv1D(
        filters=filters,
        kernel_size=kernel_size,
        activation='relu',
        padding='same',
        kernel_regularizer=l2(1e-4),
        name='conv1d_layer'
    )(x)

    # Camada de Pooling
    x = MaxPooling1D(pool_size=pool_size, padding='same', name='maxpool_layer')(x)

    # Camada LSTM Bidirecional
    x = Bidirectional(LSTM(
        units=units,
        return_sequences=True,
        activation='tanh',
        dropout=dropout_rate,
        kernel_regularizer=l2(1e-3),
        name='bidirectional_lstm_layer'
    ))(x)
    x = LayerNormalization(name='lstm_norm')(x)

    # Projetar a saída da LSTM bidirecional de volta para 'units' dimensões
    encoder_proj = Dense(units, activation='relu', name='encoder_projection')(x)

    # Encoder State (última saída de LSTM)
    encoder_state = encoder_proj[:, -1, :]  # Forma: (batch_size, units)

    # Decoder Input
    decoder_inputs = RepeatVector(1, name='repeat_vector')(encoder_state)  # Forma: (batch_size, 1, units)
    decoder_outputs = decoder_inputs

    # Camada LSTM no Decoder
    decoder_outputs = LSTM(
        units=units,
        return_sequences=True,
        activation='tanh',
        dropout=dropout_rate,
        kernel_regularizer=l2(1e-3),
        name='decoder_lstm_layer'
    )(decoder_outputs)
    decoder_outputs = LayerNormalization(name='decoder_lstm_norm')(decoder_outputs)

    # Atenção MultiCabeça
    attention_output = MultiHeadAttention(
        num_heads=num_heads,
        key_dim=units,  # key_dim agora corresponde a 'units'
        name='multi_head_attention'
    )(
        query=decoder_outputs,  # (batch_size, 1, units)
        key=encoder_proj,        # (batch_size, sequence_length, units)
        value=encoder_proj       # (batch_size, sequence_length, units)
    )
    
    # Aplicar Dropout na saída da atenção
    attention_output = Dropout(dropout_rate, name='attention_dropout')(attention_output)
    
    # Concatenar a saída da atenção com o decoder_outputs
    decoder_concat_input = Concatenate(axis=-1, name='concat_attention')([decoder_outputs, attention_output])  # (batch_size, 1, units + units)

    # Camada de Saída
    outputs = TimeDistributed(Dense(1, activation='linear'), name='output_layer')(decoder_concat_input)  # (batch_size, 1, 1)

    # Definição do Modelo
    model = Model(inputs=encoder_inputs, outputs=outputs, name='Wind_Speed_Predictor_CNN_BiLSTM_Attention')

    # Compilação do Modelo com Função de Loss Personalizada
    model.compile(
        optimizer=optimizer,
        loss=custom_loss,
        metrics=[
            tf.keras.metrics.RootMeanSquaredError(name='rmse'),
            tf.keras.metrics.MeanAbsoluteError(name='mae')
        ]
    )

    return model

# %%
# Definir o tuner usando Hyperband
tuner = Hyperband(
    build_model_cnn,
    objective='val_loss',  # Otimizar a perda de validação
    max_epochs=20,
    factor=3,
    directory='tuner_dir',
    project_name='bidirectional_lstm_tuning_updated'
)

# %%
# Callback para parada antecipada
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# %%
# Iniciar a busca de hiperparâmetros
tuner.search(
    X_train,
    y_train,
    epochs=50,
    validation_split=0.2,
    callbacks=[early_stopping]
)

# %%
# Obter os melhores hiperparâmetros
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"""
Melhores hiperparâmetros encontrados:
- Filtros: {best_hps.get('filters')}
- Tamanho do Kernel: {best_hps.get('kernel_size')}
- Tamanho do Pooling: {best_hps.get('pool_size')}
- Unidades LSTM: {best_hps.get('units')}
- Taxa de Aprendizado: {best_hps.get('learning_rate')}
- Taxa de Dropout: {best_hps.get('dropout_rate')}
- Otimizador: {best_hps.get('optimizer')}
- Número de Cabeças de Atenção: {best_hps.get('num_heads')}
""")

# %%
# Construir o melhor modelo com os hiperparâmetros encontrados
model = tuner.hypermodel.build(best_hps)

# Treinar o melhor modelo
history = model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stopping]
)

# %%
# Salvar o modelo final
model_save_path_final = 'best_bidirectional_lstm_model.h5.keras'
model.save(model_save_path_final)
print(f"Modelo salvo em: {model_save_path_final}")

# %%
# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test)

# Inversão da padronização
y_pred_inv = y_pred * (ws100_max - ws100_min) + ws100_min
y_test_inv = y_test * (ws100_max - ws100_min) + ws100_min

# Converter para classes: 1 se y < 6 m/s, 0 caso contrário
y_pred_class = (y_pred_inv < 6.0).astype(int)
y_test_class = (y_test_inv < 6.0).astype(int)

# Avaliação das previsões
mae = mean_absolute_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
mape = np.mean(np.abs((y_test_inv - y_pred_inv) / np.maximum(y_test_inv, 1e-6))) * 100  # Evitar divisão por zero

print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAPE: {mape:.2f}%\n")

# Relatório de Classificação
print("Relatório de Classificação:")
print(classification_report(y_test_class, y_pred_class))

# Plotagem das previsões vs valores reais focando em y < 6 m/s
indices_below_6 = np.where(y_test_inv < 6.0)[0]

plt.figure(figsize=(12,6))
plt.plot(indices_below_6, y_test_inv[indices_below_6], label='Valor Real')
plt.plot(indices_below_6, y_pred_inv[indices_below_6], label='Previsão')
plt.legend()
plt.title('Comparação entre Valores Reais e Previstos para y < 6 m/s')
plt.xlabel('Amostras')
plt.ylabel('Velocidade do Vento a 100 metros')
plt.show()


Trial 26 Complete [00h 01m 07s]
val_loss: 8.176505088806152

Best val_loss So Far: 4.493888854980469
Total elapsed time: 00h 15m 30s

Melhores hiperparâmetros encontrados:
- Filtros: 208
- Tamanho do Kernel: 5
- Tamanho do Pooling: 3
- Unidades LSTM: 192
- Taxa de Aprendizado: 0.0001
- Taxa de Dropout: 0.2
- Otimizador: rmsprop
- Número de Cabeças de Atenção: 4

Epoch 1/100
81/81 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - loss: 7.7189 - mae: 0.5054 - rmse: 0.6849 - val_loss: 7.1361 - val_mae: 0.2069 - val_rmse: 0.2527
Epoch 2/100
81/81 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 3.3214 - mae: 0.1766 - rmse: 0.2197 - val_loss: 5.9568 - val_mae: 0.1671 - val_rmse: 0.2008
Epoch 3/100
81/81 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 2.9239 - mae: 0.1419 - rmse: 0.1755 - val_loss: 10.3118 - val_mae: 0.2241 - val_rmse: 0.2760
Epoch 4/100
81/81 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 2.5406 - mae: 0.1191 - rmse: 0.1502 - val_loss: 6.9132 - val_mae: 0.1602 - val_rmse: 0.1952
Epoch 5/100
81/81 ━━━━━━━━

Modelo salvo em: best_bidirectional_lstm_model.h5
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step


ValueError: Found array with dim 3. None expected <= 2.

In [19]:
# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test)

# Inversão da padronização
y_pred_inv = y_pred * (ws100_max - ws100_min) + ws100_min  # Forma: (batch_size, 1, 1)
y_test_inv = y_test * (ws100_max - ws100_min) + ws100_min  # Forma: (batch_size, 1)

# Achatar os arrays para 1D
y_pred_inv = y_pred_inv.reshape(-1)
y_test_inv = y_test_inv.reshape(-1)

# Converter para classes: 1 se y < 6 m/s, 0 caso contrário
y_pred_class = (y_pred_inv < 6.0).astype(int)
y_test_class = (y_test_inv < 6.0).astype(int)

# Verificar valores únicos
print("Valores únicos em y_test_class:", np.unique(y_test_class))
print("Valores únicos em y_pred_class:", np.unique(y_pred_class))

# Avaliação das previsões
mae = mean_absolute_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
mape = np.mean(np.abs((y_test_inv - y_pred_inv) / np.maximum(y_test_inv, 1e-6))) * 100  # Evitar divisão por zero

print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAPE: {mape:.2f}%\n")

# Relatório de Classificação
print("Relatório de Classificação:")
print(classification_report(y_test_class, y_pred_class))


46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Valores únicos em y_test_class: [0 1]
Valores únicos em y_pred_class: [0 1]
MAE: 1.4868
RMSE: 1.9263
MAPE: 18.69%

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.94      0.90      0.92      1350
           1       0.27      0.41      0.33       122

    accuracy                           0.86      1472
   macro avg       0.61      0.66      0.62      1472
weighted avg       0.89      0.86      0.87      1472

